In [25]:
import pandas as pd
import numpy as np
import math
import re
from IPython.display import display

In [28]:
# 打开文件
f = open('HW2.txt')
# 读取文章，并删除每行结尾的换行符
doc = pd.Series(f.read().splitlines())
 
# print(doc)

# 转换为小写，并使用正则表达式进行切割
doc = doc.apply(lambda x: re.split('[^a-zA-Z-]', x.lower()))
# 删除空串
for llist in doc:
    while '' in llist:
        llist.remove('')
print(doc)

0     [adaptive, pairwise, preference, learning, for...
1     [hgmf, hierarchical, group, matrix, factorizat...
2     [adaptive, bayesian, personalized, ranking, fo...
3     [compressed, knowledge, transfer, via, factori...
4     [a, survey, of, transfer, learning, for, colla...
5     [mixed, factorization, for, collaborative, rec...
6     [transfer, learning, for, heterogeneous, one-c...
7     [group, bayesian, personalized, ranking, with,...
8     [transfer, learning, for, semi-supervised, col...
9       [transfer, learning, for, behavior, prediction]
10    [toccf, time-aware, one-class, collaborative, ...
11    [rbpr, role-based, bayesian, personalized, ran...
12    [hybrid, one-class, collaborative, filtering, ...
13    [mixed, similarity, learning, for, recommendat...
14    [collaborative, recommendation, with, multicla...
15         [transfer, learning, for, behavior, ranking]
16    [transfer, learning, from, app, domain, to, ne...
17    [k-cofi, modeling, k-granularity, preferen

In [29]:
hashtable = {}
for doc_index, llist in enumerate(doc):
    for term_index, term in enumerate(llist):
        # 词项出现过
        if term in hashtable:
            # 词项在doc_index中出现过
            if doc_index+1 in hashtable[term]:
                hashtable[term][doc_index+1].append(term_index+1)
             # 词项在doc_index中第一次出现
            else:
                hashtable[term][doc_index+1] = [term_index+1]
        # 词项第一次出现
        else:
            hashtable[term] = {doc_index+1: [term_index+1]}

for term in hashtable:
    print(term)
    for doc_index in hashtable[term]:
        print('\t', doc_index, ': ', hashtable[term][doc_index])


adaptive
	 1 :  [1]
	 3 :  [1]
	 33 :  [1]
pairwise
	 1 :  [2]
	 20 :  [8]
	 36 :  [2]
	 54 :  [8]
	 58 :  [6]
preference
	 1 :  [3]
	 15 :  [5]
	 18 :  [4]
	 19 :  [6]
	 23 :  [7]
	 36 :  [3]
	 40 :  [6]
	 41 :  [6]
	 57 :  [3]
learning
	 1 :  [4]
	 5 :  [5]
	 7 :  [2]
	 9 :  [2]
	 10 :  [2]
	 14 :  [3]
	 16 :  [2]
	 17 :  [2]
	 19 :  [7]
	 20 :  [7]
	 25 :  [3]
	 30 :  [3]
	 32 :  [4]
	 33 :  [3]
	 36 :  [4]
	 41 :  [7]
	 47 :  [3]
	 49 :  [2]
	 54 :  [7]
	 55 :  [3]
	 56 :  [2]
	 58 :  [5]
for
	 1 :  [5]
	 2 :  [6]
	 3 :  [5]
	 4 :  [7]
	 5 :  [6]
	 6 :  [3]
	 7 :  [3]
	 8 :  [8]
	 9 :  [3]
	 10 :  [3]
	 12 :  [6]
	 13 :  [5]
	 14 :  [4]
	 16 :  [3]
	 17 :  [9]
	 21 :  [5]
	 22 :  [7]
	 23 :  [9]
	 24 :  [4]
	 25 :  [4]
	 27 :  [5]
	 29 :  [4]
	 30 :  [4]
	 31 :  [6]
	 32 :  [5]
	 33 :  [4]
	 34 :  [6]
	 35 :  [9]
	 36 :  [5]
	 37 :  [5]
	 38 :  [9]
	 39 :  [5]
	 43 :  [5]
	 45 :  [5]
	 46 :  [6]
	 47 :  [4]
	 49 :  [6]
	 52 :  [4]
	 55 :  [4]
	 57 :  [8]
	 59 :  [4]
	 60 :  [4]
col

In [30]:
def positionalIntersect(list1, list2, k):
    ans = []
    ranges = []
    if k[0] == '+':
        ranges = [0, int(k[1])]
    elif k[0] == '-':
        ranges = [-int(k[1]), 0]
    else:
        ranges = [-int(k), int(k)]
    p_docID1, p_docID2 = 0, 0
    while p_docID1 < len(list1) and p_docID2 < len(list2):
        # 首先查找两个列表共同的DocID
        docID1, docID2 = [*list1][p_docID1], [*list2][p_docID2]
        if docID1 == docID2:
            l = []
            l1, l2 = list1[docID1], list2[docID2]
            p_pos1, p_pos2 = 0, 0
            # 二重循环遍历term出现的位置并比较
            while p_pos1 < len(l1):
                while p_pos2 < len(l2):
                    # 如果pos1 在符合条件的范围内，则加入临时答案列表l
                    if l1[p_pos1] in range(l2[p_pos2]+ranges[0], l2[p_pos2]+ranges[1]+1):
                        l.append(l2[p_pos2])
                    p_pos2 += 1
                # 格式化临时答案列表
                # 答案为三元组<文档ID， 词项在p1中的位置， 词项在p2中的位置>
                for pos2 in l:
                    ans.append([docID1, l1[p_pos1], pos2])
                p_pos1 += 1
            p_docID1 += 1
            p_docID2 += 1
        elif docID1 < docID2:
            p_docID1 += 1
        else:
            p_docID2 += 1

    return ans


In [48]:
str1, str2, k = 'ranking', 'filtering', '-4'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' ranking ']:  {3: [4], 8: [4], 12: [5], 16: [5], 27: [4], 28: [3], 57: [7]}
hashtable[' filtering ']:  {7: [7], 8: [11], 11: [5], 12: [10], 13: [4], 18: [8], 19: [3], 22: [6], 24: [8], 25: [7], 26: [5], 27: [8], 30: [8], 33: [8], 36: [9], 37: [8], 38: [3], 41: [3], 42: [7], 46: [9], 52: [8], 54: [2], 55: [6], 56: [6], 57: [11], 58: [3]}


[[27, 4, 8], [57, 7, 11]]

In [51]:
str1, str2, k = 'ranking', 'filtering', '-5'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' ranking ']:  {3: [4], 8: [4], 12: [5], 16: [5], 27: [4], 28: [3], 57: [7]}
hashtable[' filtering ']:  {7: [7], 8: [11], 11: [5], 12: [10], 13: [4], 18: [8], 19: [3], 22: [6], 24: [8], 25: [7], 26: [5], 27: [8], 30: [8], 33: [8], 36: [9], 37: [8], 38: [3], 41: [3], 42: [7], 46: [9], 52: [8], 54: [2], 55: [6], 56: [6], 57: [11], 58: [3]}


[[12, 5, 10], [27, 4, 8], [57, 7, 11]]

In [52]:
str1, str2, k = 'ranking', 'filtering', '-6'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' ranking ']:  {3: [4], 8: [4], 12: [5], 16: [5], 27: [4], 28: [3], 57: [7]}
hashtable[' filtering ']:  {7: [7], 8: [11], 11: [5], 12: [10], 13: [4], 18: [8], 19: [3], 22: [6], 24: [8], 25: [7], 26: [5], 27: [8], 30: [8], 33: [8], 36: [9], 37: [8], 38: [3], 41: [3], 42: [7], 46: [9], 52: [8], 54: [2], 55: [6], 56: [6], 57: [11], 58: [3]}


[[12, 5, 10], [27, 4, 8], [57, 7, 11]]

In [53]:
str1, str2, k = 'ranking', 'filtering', '-7'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' ranking ']:  {3: [4], 8: [4], 12: [5], 16: [5], 27: [4], 28: [3], 57: [7]}
hashtable[' filtering ']:  {7: [7], 8: [11], 11: [5], 12: [10], 13: [4], 18: [8], 19: [3], 22: [6], 24: [8], 25: [7], 26: [5], 27: [8], 30: [8], 33: [8], 36: [9], 37: [8], 38: [3], 41: [3], 42: [7], 46: [9], 52: [8], 54: [2], 55: [6], 56: [6], 57: [11], 58: [3]}


[[8, 4, 11], [12, 5, 10], [27, 4, 8], [57, 7, 11]]

In [57]:
str1, str2, k = 'heterogeneous', 'learning', '+2'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' heterogeneous ']:  {3: [6], 4: [8], 6: [7], 7: [4], 12: [7], 24: [5], 30: [5], 32: [8], 33: [5], 36: [6], 40: [4], 42: [4], 45: [2], 46: [7], 52: [5], 53: [6], 55: [8], 56: [4]}
hashtable[' learning ']:  {1: [4], 5: [5], 7: [2], 9: [2], 10: [2], 14: [3], 16: [2], 17: [2], 19: [7], 20: [7], 25: [3], 30: [3], 32: [4], 33: [3], 36: [4], 41: [7], 47: [3], 49: [2], 54: [7], 55: [3], 56: [2], 58: [5]}


[[7, 4, 2], [30, 5, 3], [33, 5, 3], [36, 6, 4], [56, 4, 2]]

In [59]:
str1, str2, k = 'recommendation', 'bias', '2'
print('hashtable[\'', str1, '\']: ', hashtable[str1])
print('hashtable[\'', str2, '\']: ', hashtable[str2])
positionalIntersect(hashtable[str1], hashtable[str2], k)

hashtable[' recommendation ']:  {1: [7], 2: [8], 4: [10], 5: [8], 6: [5], 9: [6], 13: [7], 14: [5], 15: [2], 17: [12], 20: [2], 21: [7], 26: [2], 29: [6], 31: [8], 32: [6], 34: [8], 35: [11], 38: [11], 39: [7], 43: [7], 44: [3], 45: [6], 47: [6], 48: [3], 49: [5], 50: [5], 51: [4], 53: [4], 59: [6], 60: [6]}
hashtable[' bias ']:  {49: [7], 50: [3]}


[[49, 5, 7], [50, 5, 3]]